In [20]:
workflows = dict()
objects = []

rules_finished = False
with open('example_input') as f:
    for line in f:
        if len(line) == 1 and rules_finished is False:
            rules_finished = True
            continue
        if rules_finished is False:
            line = line.strip()
            name = line.split('{')[0]
            rules = line.split('{')[1][:-1]
            rules = rules.split(',')
            rules_to_save = []
            for rule in rules:
                new_rule = rule.split(":")
                if len(new_rule) > 1:
                    com_1 = new_rule[0][0]
                    com_2 = new_rule[0][1]
                    com_3 = new_rule[0][2:]
                    com_4 = new_rule[1]
                    rules_to_save.append([com_1, com_2, com_3, com_4])
                else:
                    rules_to_save.append([new_rule[0]])
            workflows[name] = rules_to_save   

        if rules_finished is True:
            line = line.strip()[1:-1].split(",")
            new_object = dict()
            for metric in line:
                var = metric[0]
                value = int(metric[2:])
                new_object[var] = value
            objects.append(new_object)

x_steps = [1, 4000]
m_steps = [1, 4000]
a_steps = [1, 4000]
s_steps = [1, 4000]

for flow in workflows:
    for rule in workflows[flow]:
        if rule[0] == 'x':
            if rule[1] == '<':
                x_steps.append(int(rule[2])-1)
                x_steps.append(int(rule[2]))
            if rule[1] == '>':
                x_steps.append(int(rule[2]))
                x_steps.append(int(rule[2])+1)
        if rule[0] == 'm':
            if rule[1] == '<':
                m_steps.append(int(rule[2])-1)
                m_steps.append(int(rule[2]))
            if rule[1] == '>':
                m_steps.append(int(rule[2]))
                m_steps.append(int(rule[2])+1)
        if rule[0] == 'a':
            if rule[1] == '<':
                a_steps.append(int(rule[2])-1)
                a_steps.append(int(rule[2]))
            if rule[1] == '>':
                a_steps.append(int(rule[2]))
                a_steps.append(int(rule[2])+1)
        if rule[0] == 's':
            if rule[1] == '<':
                s_steps.append(int(rule[2])-1)
                s_steps.append(int(rule[2]))
            if rule[1] == '>':
                s_steps.append(int(rule[2]))
                s_steps.append(int(rule[2])+1)

x_steps = sorted(x_steps)
m_steps = sorted(m_steps)
a_steps = sorted(a_steps)
s_steps = sorted(s_steps)

print("x_steps", x_steps)
print("m_steps", m_steps)
print("a_steps", a_steps)
print("s_steps", s_steps)

total = 0

for i in range(len(x_steps)-1):
    for j in range(len(m_steps)-1):
        for k in range(len(a_steps)-1):
            for l in range(len(s_steps)-1):
                total_case = 0
                x_mult = (x_steps[i+1]) - (x_steps[i])
                m_mult = (m_steps[j+1]) - (m_steps[j])
                a_mult = (a_steps[k+1]) - (a_steps[k])
                s_mult = (s_steps[l+1]) - (s_steps[l])
                object = dict()
                object['x'] = x_steps[i]
                object['m'] = m_steps[j]
                object['a'] = a_steps[k]
                object['s'] = s_steps[l]
                next_workflow = "in"
                print(x_mult)
                while True:
                    if next_workflow in ['A', 'R']:
                        if next_workflow == 'A':
                            total_case += 1
                        break
                    for rule in workflows[next_workflow]:
                        next_workflow = None
                        if len(rule) == 1:
                            next_workflow = rule[0]
                            break
                        if rule[0] in object:
                            val = object[rule[0]]
                            threshold = int(rule[2])
                            if rule[1] == '<':
                                if val < threshold:
                                    next_workflow = rule[3]
                                    break
                            if rule[1] == '>':
                                if val > threshold:
                                    next_workflow = rule[3]
                                    break
                total += total_case * x_mult * m_mult * a_mult * s_mult
                    
print(total)

x_steps [1, 1415, 1416, 2440, 2441, 2662, 2663, 4000]
m_steps [1, 838, 839, 1548, 1549, 1800, 1801, 2090, 2091, 4000]
a_steps [1, 1716, 1717, 2005, 2006, 3333, 3334, 4000]
s_steps [1, 536, 537, 1350, 1351, 2770, 2771, 3448, 3449, 4000]
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414
1414